# Glacier bed inversion in OGGM

So far, we have only looked at the glacier surface: first by defining the surface geometry of our flowline, and then by defining and calibrating the surface mass balance. In this session, we will focus on how to define the glacier bed, which is necessary for running a dynamic simulation in the next session. However, since there are no direct observations available on a global scale, defining the glacier bed is a challenging task that requires some assumptions.

Let’s dive in!

## OGGM setup

First, we set up OGGM as usual. We still need to start from the level 2 preprocessed directories, as we are not yet at level 3. However, I will redo some tasks from previous sessions to bring the glacier directory to the stage we need for today’s work.

In [ ]:
# first we import the packages we need for this session
from oggm import cfg, utils, workflow, tasks, graphics 
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cfg.initialize()

# define working directory
path = 'inversion_working_dir'
utils.mkdir(path, reset=False)  # if you set reset=True, everything will be deleted and you can start from a fresh state
cfg.PATHS['working_dir'] = path

In [ ]:
# select the glacier of your choice, you can use the Glims viewer from the first session
rgi_ids = ['RGI60-11.00897']  # Hintereisferner

# we load the outline data from the oggm cluster
prepro_base_url_L2 = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/elev_bands/'
gdirs = workflow.init_glacier_directories(rgi_ids,
                                          from_prepro_level=2,  # here we select level 2
                                          prepro_base_url=prepro_base_url_L2,
                                          prepro_border=80,  # could be 10, 80, 160 or 240
                                         )
gdir = gdirs[0]  # for convenience, we define a single variable for our glacier

# redo some tasks of previous sessions to get everything we need for the inversion
workflow.execute_entity_task(tasks.compute_downstream_bedshape, gdirs);
workflow.execute_entity_task(tasks.process_climate_data, gdirs);
workflow.execute_entity_task(tasks.mb_calibration_from_geodetic_mb,
                             gdirs,
                             informed_threestep=True,
                            );

## The actual inversion

To get started, please read through the description of the [Bed inversion section in the OGGM documentation](https://docs.oggm.org/en/stable/inversion.html), up to the paragraph titled "Calibration". This information will help you follow along with the tasks discussed below.

Once you have finished reading, we will begin with the actual inversion.

## Calculation of the apparent mass balance

<div class="alert alert-warning">
    <b>Task</b>:
    <ul>
        <li>What is the <b>apparent mass balance</b>?</li>
        <li>What is the difference between the <b>apparent mass balance</b> and the <b>surface mass balance</b> we studied in the last session?</li>
    </ul>
</div>

Your answer here:

Calculating the apparent mass balance is simply a matter of running another task:

In [ ]:
workflow.execute_entity_task(tasks.apparent_mb_from_any_mb, gdirs);

We can take a look at the diagnostics to see what we have actually calculated:

In [ ]:
gdir.get_diagnostics()['apparent_mb_from_any_mb_residual']

Let’s also take a look at the mean specific surface mass balance for the period 2000 to 2020:

In [ ]:
from oggm.core import massbalance

# defining the mb model the same way as in the last session
mb_model = massbalance.MonthlyTIModel(gdir)

# define the period we want to look at
mb_years = np.arange(2000, 2020, 1)

# get the surface flowline geometery, which we defined two sessions ago
fls = gdir.read_pickle('inversion_flowlines')

# and calculate the mean specific surface mass balance
np.mean(mb_model.get_specific_mb(fls=fls, year=mb_years))

<div class="alert alert-warning">
    <b>Task</b>: Why are these two numbers the same (except for their sign)? What are their units? (Hint: Check the definition of the equilibrium assumption in the documentation.)
</div>

Your answer here:

## The actual bed inversion

After calculating the apparent mass balance, we can use it to perform the actual inversion. Once the inversion is completed, we combine the inverted glacier bed with the downstream line we defined two sessions ago by calling `tasks.init_present_time_glacier`.

In [ ]:
from oggm import global_tasks

# the inverison can be done at the same time for mutliple glaciers,
# why this is different to the other tasks we have seen so far will
# get clear in the upcoming sections
global_tasks.inversion_tasks(gdirs,
                             filter_inversion_output=False  # for what this filter is used is explained below
                            )

# finally we put together our glacier bed with the downstream line we defined two sessions ago
workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs,
                             filesuffix='_no_filter_no_calibration');

Let's have a look at it:

In [ ]:
fls_no_filter_no_calibration = gdir.read_pickle('model_flowlines', filesuffix='_no_filter_no_calibration')
graphics.plot_modeloutput_section(fls_no_filter_no_calibration)

<div class="alert alert-warning">
    <b>Task</b>: Why do we see a large overdeepening at the terminus? (Hint: Review how the inversion is performed in the documentation and think again about the equilibrium assumption.)
</div>

Your anwser here:

Since we are aware of the overdeepening problem, we can fix it by applying a filter at the terminus. Let’s redo the inversion with the filter applied and then check the difference:

In [ ]:
# let's redo the inversion, by using a filter at the terminus
global_tasks.inversion_tasks(gdirs,
                             filter_inversion_output=True
                            )
workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs,
                             filesuffix='_no_calibration');

In [ ]:
fls_no_calibration = gdir.read_pickle('model_flowlines', filesuffix='_no_calibration')
graphics.plot_modeloutput_section(fls_no_calibration)

After completing the inversion, we can take a look at our calculated volume and compare it to another volume estimate, specifically, the consensus estimate from [Farinotti et al. (2019)](https://doi.org/10.1038/s41561-019-0300-3).

In [ ]:
# Define a function to compare the modelled volume with the consensus estimate
def print_modelled_and_estimated_volume(gdir):
    # Get the modelled volume from the bed inversion (convert from m³ to km³)
    model_volume = tasks.get_inversion_volume(gdir) * 1e-9  # km³

    # Get the consensus estimate volume (convert from m³ to km³)
    estimated_volume = pd.read_hdf(utils.get_demo_file('rgi62_itmix_df.h5')).loc[gdir.rgi_id]['vol_itmix_m3'] * 1e-9  # km³
    
    print(f"Estimated volume: {estimated_volume:.3f} km³")
    print(f'Modelled volume: {model_volume:.3f} km³')

print_modelled_and_estimated_volume(gdir)

As you can see, we are quite far off. The reason for this is that we used the default Glen A parameter during the inversion:

In [ ]:
gdir.get_diagnostics()['inversion_glen_a']

You can redo the inversion using your own Glen A parameter by setting the glen_a keyword argument.

Can you find a better match to the estimated volume?

In [ ]:
# let's redo the inversion, by using a filter at the terminus
global_tasks.inversion_tasks(gdirs,
                             filter_inversion_output=True,
                             # here you can play around by providing your own Glen A paramter
                             glen_a=cfg.PARAMS['glen_a'] * 1,
                            )
workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs,
                             filesuffix='_your_glen_a');

<div class="alert alert-warning">
    <b>Task</b>: What does the Glen A parameter describe? In which direction do you think we need to change it to bring the modelled volume closer to the estimated one? (Hint: Take a look at the <a href="https://docs.oggm.org/en/stable/ice-dynamics.html#ice-flow">OGGM documentation</a> and the plot in <a href="https://tutorials.oggm.org/stable/notebooks/tutorials/inversion.html#sensitivity-analysis">this section of the OGGM tutorials</a>.)
</div>

Your answer here:

### Bed inversion with calibration to estimated volume

Fortunately, OGGM has a functionality that allows us to take the estimated volume into account and calibrate the Glen A parameter for better agreement:

In [ ]:
# when we use all glaciers, no Glen A could be found within the range [0.1,10] that would match the consensus estimate
# usually, this is applied on larger regions where this error should not occur ! 
workflow.calibrate_inversion_from_consensus(gdirs, filter_inversion_output=True);
workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs);

In the default OGGM workflow, this calibration is not done for each glacier individually, but rather for an entire region.
The reason is that the consensus volume estimate is still just an estimate, not a direct observation. We assume larger errors for individual glaciers, but that the estimate becomes more accurate when averaged over large regions.

Now, let’s check again how well we match the estimated volume:

In [ ]:
print_modelled_and_estimated_volume(gdir)

Great, now we have a perfect match!

Let’s take a look at the Glen A parameter that was used. Did it change in the direction you expected?

In [ ]:
gdir.get_diagnostics()['inversion_glen_a']

Finally, let’s plot the three glacier beds we created today side by side (if you want you can also add your own bed inversion here):

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(6,10), gridspec_kw={'hspace': 0.6})

# our intial approach, without a filter at the terminus and no calibration
graphics.plot_modeloutput_section(fls_no_filter_no_calibration, ax=axs[0])
axs[0].set_title('  _no_filter_no_calibration', loc='left')

# our secend try, with a filter at the terminus, but no calibration
graphics.plot_modeloutput_section(fls_no_calibration, ax=axs[1])
axs[1].set_title('  _no_calibration', loc='left')

# our final bed inversion, with a filter and with a calibration
fls_oggm = gdir.read_pickle('model_flowlines')
graphics.plot_modeloutput_section(fls_oggm, ax=axs[2])
axs[2].set_title('  OGGM default', loc='left')

<div class="alert alert-warning">
    <b>Task</b>: Summarize the differences between the three glacier geometries in your own words.
</div>

Your answer here:

<div class="alert alert-warning">
    <b>Bonus-task</b>: Calculate the differences between the flowline thicknesses of the different glacier beds and compare them.
</div>

In [ ]:
# Hint you can get access to the thickness with the following code
fls_oggm[0].thick

## Bonus: Distributed thickness

There are also some functionalities available to redistribute the flowline thickness back onto the grid:

In [ ]:
# Distribute flowline thickness to the grid
workflow.execute_entity_task(tasks.distribute_thickness_per_altitude, gdirs);

Before we look at the results, let's also add some observations for comparison, starting with the consensus estimate from [Farinotti et al. (2019)](https://doi.org/10.1038/s41561-019-0300-3).

In [ ]:
from oggm.shop import bedtopo
workflow.execute_entity_task(bedtopo.add_consensus_thickness, gdirs);

And now, let's add a velocity-based estimate from [Millan et al. (2022)](https://doi.org/10.1038/s41561-021-00885-z):

In [ ]:
from oggm.shop import millan22
workflow.execute_entity_task(millan22.thickness_to_gdir, gdirs);

Okay, let’s load the data and compare it in a plot:

In [ ]:
# load all gridded data, this is the grid we defined in the second session
with xr.open_dataset(gdir.get_filepath('gridded_data')) as ds:
    ds = ds.load()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10,8), gridspec_kw={'hspace': 0.25})

# the oggm output
ax_oggm = axs[0, 0]
ds.distributed_thickness.plot(ax=ax_oggm);
ax_oggm.set_title('OGGM')

# the consensus estimate
ax_consensus = axs[0, 1]
ds.consensus_ice_thickness.plot(ax=ax_consensus)
ax_consensus.set_title('Farinotti et al. (2019)')

# the Millan estimate
ax_millan = axs[1, 0]
ds.where(ds.glacier_mask).millan_ice_thickness.plot(ax=ax_millan)
ax_millan.set_title('Millan et al. (2022)')


<div class="alert alert-warning">
    <b>Task</b>: What differences do you notice?
</div>

Your answer here:

## Recap

- We need to assume equilibrium during the inversion process.
- This assumption can lead to overdeepenings at the terminus for retreating glaciers.
- We can use a total volume estimate to calibrate the glacier flow parameter Glen A.